In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import torch
import torch.nn as nn
import torch_geometric as tg
import torch.nn.functional as F
from torch.nn import init
import pdb
from torch.utils.data import Dataset, DataLoader, random_split
from torch_geometric.data import Data
import torch.optim as optim
import scipy.interpolate as interp
from IPython.display import clear_output
import matplotlib.ticker as mticker

In [2]:
def open_data(file_path):
    file = open(file_path,"rb")
    raw_data = pickle.load(file)  
    return raw_data

In [3]:
def save_data(data, file_path):
    with open(file_path , 'wb') as f:
        pickle.dump(data,f)
        f.close()

In [4]:
class TimeSeriesDataset(Dataset):
    def __init__(self, inputs, targets, dist, dist_arg, masks, min_vals, max_vals, mask_dist_max, class_labels):
        self.inputs = inputs
        self.targets = targets
        self.dist = dist
        self.dist_arg = dist_arg
        self.masks = masks
        self.min_vals = min_vals
        self.max_vals = max_vals
        self.mask_dist_max = mask_dist_max
        self.class_labels = class_labels
    
    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        # 获取输入和对应的目标数据
        x = self.inputs[idx]
        y = self.targets[idx]
        d_ = self.dist[idx]
        d_arg = self.dist_arg[idx]
        mask = self.masks[idx]
        min_ = self.min_vals[idx]
        max_ = self.max_vals[idx]
        mask_d_ = self.mask_dist_max[idx]
        class_ = self.class_labels[idx]
        return x, y, d_, d_arg, mask, min_, max_, mask_d_, class_

In [5]:
class NonLinearOperate(nn.Module):
    def __init__(self, input_dimen, hidden_dimen, output_dimen):
        super(NonLinearOperate, self).__init__()  #类NonLinearLayer继承父类nn.Module的初始化方法
        self.layer_1 = nn.Linear(input_dimen, hidden_dimen)
        self.layer_2 = nn.Linear(hidden_dimen, output_dimen)
        self.acti_func = nn.ReLU()
        for m in self.modules():#遍历所有子模块
        #Check if each sub-module is an example of the class nn.Linear
            if isinstance(m, nn.Linear):
                m.weight.data = init.kaiming_normal_(m.weight.data, nonlinearity='relu')
                if m.bias is not None:
                    m.bias.data = init.constant_(m.bias.data, 0.0)
            
        
    def forward(self, x):
        x = self.layer_1(x)
        x = self.acti_func(x)
        x = self.layer_2(x)
        x = self.acti_func(x)
        
        return x

In [6]:
class PGNN_Layer(nn.Module):
    def __init__(self, hidden_dimen, output_dimen, anchor_num, drop_out = True):
        super(PGNN_Layer, self).__init__()
        self.drop_out = drop_out
        self.output_dimen = output_dimen
        self.acti_func = nn.LeakyReLU()
        self.linear_hidden = nn.Linear(2 * anchor_num, hidden_dimen)
        self.out_layer = nn.Linear(hidden_dimen, output_dimen)
        
        self.linear_structure = nn.Linear(hidden_dimen, 1)

        
        for m in self.modules():
            if isinstance(m, nn.Linear):
                m.weight.data = init.kaiming_uniform_(m.weight.data, nonlinearity='leaky_relu')
                if m.bias is not None:
                    m.bias.data = init.constant_(m.bias.data, 0.0)                
                    
    def forward(self, node_features, dists_max, dists_argmax, mask_dist_max):
        # node feature SHAPE: Batch-size, 28, Max_node_num
        # Dist_max SHAPE: Batch-size, 28, Max_node_num, 36
        # mask_dist_max SHAPE: Batch-size, 28, Max_node_num, 36
        batch_size, num_days, max_node_num = node_features.size()
        #dists_max = self.distance_calculate(dists_max.unsqueeze(-1)).squeeze()

        indices_expanded = dists_argmax.flatten(start_dim=2)
        batch_indices = torch.arange(batch_size).view(-1, 1, 1).expand(-1, num_days, indices_expanded.shape[-1])
        day_indices = torch.arange(num_days).view(1, -1, 1).expand(batch_size, -1, indices_expanded.shape[-1]) 
        
        subset_features = node_features[batch_indices, day_indices, indices_expanded]
        
    
        #subset_features SHAPE: Batch-size, 28, Max_node_num*36
        #subset_features = torch.gather(node_features, dim=2, index=indices_expanded)


        #---------------------------------------------------------------
        subset_features = subset_features.reshape(subset_features.shape[0], subset_features.shape[1], dists_argmax.shape[2], dists_argmax.shape[3])
        #message SHAPE: Batch-size, 28, Max_node_num, 36
        messages = subset_features * dists_max * mask_dist_max
        feature_self = node_features.unsqueeze(-1).repeat(1, 1, 1, dists_max.shape[-1])
        
        #messages SHAPE: Batch-size, 28, Max_node_num * (36*2)
        messages = torch.concat((messages, feature_self), dim = -1)
        #---------------------------------------------------------------
        #INPUT DIMEN:Batch-size, 28, Max_node_num, 72
        messages = self.linear_hidden(messages)
        if self.drop_out:
            messages = F.dropout(messages, training=self.training, p=0.2)
        #SHAPE: Batch-size, 28, Max_node_num, hidden_dimen
        messages = self.acti_func(messages) 
        
        #SHAPE: Batch-size, 28, Max_node_num, Output_dimen
        output = self.out_layer(messages)
        if self.drop_out:
            output = F.dropout(output, training=self.training, p =0.2)
        output = self.acti_func(output) 
        
        output_structure = self.linear_structure(messages)
        if self.drop_out:
            output_structure = F.dropout(output_structure, training=self.training, p=0.2)
        output_structure = self.acti_func(output_structure).unsqueeze(-1)
        
        
        return output, output_structure

In [7]:
class PGNN(nn.Module):
    def __init__(self, hidden_dimen, output_dimen, anchor_num, layer_num = 1, drop_out = True):
        super(PGNN, self).__init__()
        self.drop_out = drop_out
        self.layer_num = layer_num
        self.anchor_num = anchor_num
        if self.layer_num == 1:
            self.pgnn_operate_1 = PGNN_Layer(hidden_dimen, output_dimen, anchor_num, drop_out)
        
            
        if self.layer_num > 1:
            self.pgnn_operate_1 = nn.ModuleList([PGNN_Layer(hidden_dimen, output_dimen, anchor_num, drop_out) for i in range(0, layer_num)])
           
                
        
    def forward(self, x, dist_max, dist_argmax, mask_dist_max):

        if self.layer_num == 1:
            x_position, x = self.pgnn_operate_1(x, dist_max, dist_argmax, mask_dist_max)
    
            return x_position

        
        if self.layer_num > 1:
            for i in range(self.layer_num):
                _, x = self.pgnn_operate_1[i](x, dist_max, dist_argmax, mask_dist_max)
            x_position = _   

            return x_position

In [8]:
#PGNN处理后数据形状 : Batch_num, 28, Max_node_num, output_dimen ------> (Batch_num * Max_node_num), output_dimen, 28
class CNN_1D(nn.Module):
    def __init__(self, output_dimen, hidden_channels_1, hidden_channels_2):
        super(CNN_1D, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv1d(in_channels = output_dimen, out_channels = hidden_channels_1, kernel_size=7,  stride=1, padding=0),
            nn.LeakyReLU(), #len 22 days
            nn.Conv1d(in_channels = hidden_channels_1, out_channels = hidden_channels_2, kernel_size=7,  stride=1, padding=0),
            nn.LeakyReLU(), #len 16 days
            nn.Conv1d(in_channels = hidden_channels_2, out_channels = hidden_channels_2, kernel_size = 6, stride = 2, padding=1),
            nn.LeakyReLU())  #len: 7 days

        
        for m in self.modules():
            if isinstance(m, nn.Conv1d) or isinstance(m, nn.ConvTranspose1d):
                # 对卷积层使用 Kaiming 正态初始化
                nn.init.kaiming_uniform_(m.weight.data, nonlinearity='leaky_relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
        
       
        
#PGNN处理后数据形状 : Batch_num, 28, Max_node_num, output_dimen ------> (Batch_num * Max_node_num), output_dimen, 28
    
    def forward(self, x):
        batch_size, seq_len, max_node_num, out_dim = x.size()
        x= x.permute(0, 2, 3, 1)
        x = x.flatten(start_dim=0, end_dim=1)
        #output shape: (Batch_num * Max_node_num), hidden_channels_2, 7
        out = self.conv_layer(x) 
        #----------------------------------------------------
        #output shape: (Batch_num * Max_node_num), 7, hidden_channels_2
        out = out.permute(0, 2, 1)
        #------------------------------------------------------

        #out = out.view(batch_size, max_node_num, out.shape[1], out.shape[2])
        #out = out.permute(0, 3, 1, 2)
        #out shape: Batch_num, 7, (Max_node_num,  hidden_channels_2)
        #out = out.flatten(start_dim = 2, end_dim = 3)
       
        return out

In [9]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, output_seq_len, num_nodes, drop_out = True):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers = 1, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size * output_seq_len)
        self.num_nodes = num_nodes
        self.output_seq_len = output_seq_len
        self.output_size = output_size
        self.acti_func = nn.ReLU()
        self.drop_out = drop_out
        for m in self.modules():
            if isinstance(m, nn.Linear):
                m.weight.data = init.kaiming_uniform_(m.weight.data, nonlinearity='relu')
                if m.bias is not None:
                    m.bias.data = init.constant_(m.bias.data, 0.0) 
          
    
    def forward(self, x):
        # x shape: (Batch_num * Max_node_num), 7, hidden_size
        lstm_out, _ = self.lstm(x)
        if self.drop_out:
            lstm_out = F.dropout(lstm_out, training=self.training, p=0.2)
        lstm_out = self.acti_func(lstm_out)
        # lstm_out shape: (Batch_num * Max_node_num), hidden_size 
        lstm_out = lstm_out[:, -1, :]
        out = self.fc(lstm_out)  
        out= self.acti_func(out)
        #out shape: Batch_num, Max_node_num, output_seq_len
        out = out.view(-1, self.num_nodes, self.output_seq_len).permute(0, 2, 1)
         
        return out

In [10]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, output_seq_len, num_nodes, drop_out = True):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers=1, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size * output_seq_len)
        self.num_nodes = num_nodes
        self.output_seq_len = output_seq_len
        self.output_size = output_size
        self.acti_func = nn.ReLU()
        self.drop_out = drop_out
        for m in self.modules():
            if isinstance(m, nn.Linear):
                m.weight.data = init.kaiming_uniform_(m.weight.data, nonlinearity='relu')
                if m.bias is not None:
                    m.bias.data = init.constant_(m.bias.data, 0.0)  

    
    def forward(self, x):
        # x shape: (Batch_num * Max_node_num), 7, hidden_size
        lstm_out, _ = self.rnn(x)
        if self.drop_out:
            lstm_out = F.dropout(lstm_out, training=self.training, p=0.2)
        lstm_out = self.acti_func(lstm_out)
        # lstm_out shape: (Batch_num * Max_node_num), hidden_size 
        lstm_out = lstm_out[:, -1, :]
        out = self.fc(lstm_out)  
        out= self.acti_func(out)
        #out shape: Batch_num, Max_node_num, output_seq_len
        out = out.view(-1, self.num_nodes, self.output_seq_len).permute(0, 2, 1)
         
        return out

In [11]:
#----------------------------------------------------------------------
#PGNN
anchor_num = 50
num_nodes = 1466    
hidden_dimen = 128  
output_dimen = 64
#-------------------------------------------------
#CNN
hidden_channels_1, hidden_channels_2 = 128, 64
#-------------------------------------------------
input_size = hidden_channels_2  # LSTM 输入的维度
hidden_size = 128   # LSTM 隐藏层的维度
output_seq_len = 14  # 预测14天
output_size = 1



In [12]:
lstm = LSTMModel(input_size, hidden_size, output_size, output_seq_len, num_nodes)
#lstm = RNNModel(input_size, hidden_size, output_size, output_seq_len, num_nodes)
lstm_optimizer = optim.NAdam(lstm.parameters(), lr= 4e-4, weight_decay=1e-5) #Nadam 4e-4
lstm_scheduler = optim.lr_scheduler.StepLR(lstm_optimizer, step_size=11, gamma=0.6)
#------------------------------------------------------------------------------------------
pgnn = PGNN(hidden_dimen, output_dimen, anchor_num)
cnn_pgnn_optimizer = optim.NAdam(list(pgnn.parameters()), lr= 6e-4, weight_decay=1e-5) 
cnn_pgnn_scheduler = optim.lr_scheduler.StepLR(cnn_pgnn_optimizer, step_size=11, gamma=0.55)
cnn = CNN_1D(output_dimen, hidden_channels_1, hidden_channels_2)
cnn_optimizer = optim.NAdam(list(cnn.parameters()), lr= 6e-4, weight_decay=1e-5) #Nadam 6e-4
cnn_scheduler = optim.lr_scheduler.StepLR(cnn_optimizer, step_size=11, gamma=0.55)
#----------------------------------------------------------------------------------------------------
criterion_MSE = nn.MSELoss(reduction='none')  # 使用均方误差作为损失函数
criterion_MAE = nn.L1Loss(reduction='none')

In [26]:
checkpoint = torch.load('D:/ThesisData/processed data/ModelPara/source_pgnn_cnn_lstm.pth')
pgnn.load_state_dict(checkpoint['pgnn_state_dict'])
cnn.load_state_dict(checkpoint['cnn_state_dict'])
lstm.load_state_dict(checkpoint['lstm_state_dict'])
cnn_pgnn_optimizer.load_state_dict(checkpoint['cnn_pgnn_optimizer_state_dict'])
cnn_optimizer.load_state_dict(checkpoint['cnn_optimizer_state_dict'])
lstm_optimizer.load_state_dict(checkpoint['lstm_optimizer_state_dict'])

In [13]:
train_dataset = torch.load("D:/ThesisData/processed data/SourceDomain/NEW/train_data_14days.h5")

In [14]:
vali_bangkok = torch.load("D:/ThesisData/processed data/SourceDomain/NEW/Bangkok_vali_data_14days.h5")
vali_antwerp = torch.load("D:/ThesisData/processed data/SourceDomain/NEW/Antwerp_vali_data_14days.h5")

In [15]:
test_bangkok = torch.load("D:/ThesisData/processed data/SourceDomain/NEW/Bangkok_test_data_14days.h5")
test_antwerp = torch.load("D:/ThesisData/processed data/SourceDomain/NEW/Antwerp_test_data_14days.h5")

In [16]:
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
Bangkok_vali_loader = DataLoader(vali_bangkok, batch_size=batch_size, shuffle=False)
Antwerp_vali_loader = DataLoader(vali_antwerp, batch_size=batch_size, shuffle=False)
Bangkok_test_loader = DataLoader(test_bangkok, batch_size=batch_size, shuffle=False)
Antwerp_test_loader = DataLoader(test_antwerp, batch_size=batch_size, shuffle=False)

In [17]:
epoch_num = 40

In [18]:
for h in range(epoch_num):
    lstm.train()
    pgnn.train()
    cnn.train()
    
    
    for batch_input, batch_labels, batch_dist, batch_dist_arg, batch_masks, batch_min, batch_max, batch_dist_mask, _ in train_loader:

        
        batch_input, batch_labels, batch_masks = batch_input.squeeze(), batch_labels.squeeze(), batch_masks.squeeze()
        batch_min, batch_max = batch_min.squeeze(), batch_max.squeeze()
  
        

        
        pgnn_output = pgnn(batch_input, batch_dist, batch_dist_arg, batch_dist_mask)
        cnn_output = cnn(pgnn_output)
        
        batch_outputs = lstm(cnn_output)
      
        
        loss_mse = criterion_MSE(batch_outputs, batch_labels) * batch_masks
        loss_mae = criterion_MAE(batch_outputs, batch_labels) * batch_masks
        loss_mse = loss_mse.sum() / batch_masks.sum()
        loss_mae = loss_mae.sum() / batch_masks.sum()
        
        lstm_optimizer.zero_grad()
        cnn_pgnn_optimizer.zero_grad()
        cnn_optimizer.zero_grad()

       
        loss_mse.backward()

        
        torch.nn.utils.clip_grad_norm_(lstm.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(cnn.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(pgnn.parameters(), 1.0)

        
        lstm_optimizer.step()
        cnn_pgnn_optimizer.step()
        cnn_optimizer.step()
        
        del loss_mse
        del loss_mae
        #----------------------------------------------------------------------
        inverse_outputs = batch_outputs * (batch_max - batch_min) + batch_min
        inverse_labels = batch_labels * (batch_max - batch_min) + batch_min

        
        loss_mse = criterion_MSE(inverse_outputs, inverse_labels) * batch_masks
        loss_mae = criterion_MAE(inverse_outputs, inverse_labels) * batch_masks
        
        
        
        loss_mse = loss_mse.sum() / batch_masks.sum()
        loss_mae = loss_mae.sum() / batch_masks.sum()    
        
        del batch_outputs
        del batch_labels
        del inverse_outputs
        del inverse_labels
        del batch_min
        del batch_max

    
    print(f"epoch {h + 1}, Train Set Inversed Values: MSE={loss_mse.item():.1f}, MAE={loss_mae.item():.1f}")  
    
    
#--------------------------------------------------
    lstm.eval()
    pgnn.eval()
    cnn.eval()  
    
    with torch.no_grad():

        for batch_input, batch_labels, batch_dist, batch_dist_arg, batch_masks, batch_min, batch_max, batch_dist_mask, _ in Antwerp_vali_loader:
            batch_input, batch_labels, batch_masks = batch_input.squeeze(), batch_labels.squeeze(), batch_masks.squeeze()
            batch_min, batch_max = batch_min.squeeze(), batch_max.squeeze()


            pgnn_output = pgnn(batch_input, batch_dist, batch_dist_arg, batch_dist_mask)
            cnn_output = cnn(pgnn_output)
            batch_outputs = lstm(cnn_output)           

      
            del pgnn_output
            del cnn_output


            #----------------------------------------------------------------------
            Antwerp_vali_outputs = batch_outputs * (batch_max - batch_min) + batch_min
            Antwerp_vali_labels = batch_labels * (batch_max - batch_min) + batch_min            
            #----------------------------------------------------------------------
        

            loss_mse = criterion_MSE(Antwerp_vali_outputs, Antwerp_vali_labels) * batch_masks
            loss_mae = criterion_MAE(Antwerp_vali_outputs, Antwerp_vali_labels) * batch_masks
            loss_mse = loss_mse.sum() / batch_masks.sum()
            loss_mae = loss_mae.sum() / batch_masks.sum()   

          
        print(f"----Validation Antwerp: MSE={loss_mse.item():.1f}, MAE={loss_mae.item():.1f}")


        for batch_input, batch_labels, batch_dist, batch_dist_arg, batch_masks, batch_min, batch_max, batch_dist_mask, _ in Antwerp_test_loader:
            batch_input, batch_labels, batch_masks = batch_input.squeeze(), batch_labels.squeeze(), batch_masks.squeeze()
            batch_min, batch_max = batch_min.squeeze(), batch_max.squeeze()


            pgnn_output = pgnn(batch_input, batch_dist, batch_dist_arg, batch_dist_mask)
            cnn_output = cnn(pgnn_output)
            batch_outputs = lstm(cnn_output)           

      
            del pgnn_output
            del cnn_output


            #----------------------------------------------------------------------
            Antwerp_test_outputs = batch_outputs * (batch_max - batch_min) + batch_min
            Antwerp_test_labels = batch_labels * (batch_max - batch_min) + batch_min            
            #----------------------------------------------------------------------
        

            loss_mse = criterion_MSE(Antwerp_test_outputs, Antwerp_test_labels) * batch_masks
            loss_mae = criterion_MAE(Antwerp_test_outputs, Antwerp_test_labels) * batch_masks
            loss_mse = loss_mse.sum() / batch_masks.sum()
            loss_mae = loss_mae.sum() / batch_masks.sum()   

          
        print(f"----Test Set Antwerp: MSE={loss_mse.item():.1f}, MAE={loss_mae.item():.1f}")



        
#----------------------------------------------------------------------------------------------------------------------------------------------
        for batch_input, batch_labels, batch_dist, batch_dist_arg, batch_masks, batch_min, batch_max, batch_dist_mask, _ in Bangkok_vali_loader:
            batch_input, batch_labels, batch_masks = batch_input.squeeze(), batch_labels.squeeze(), batch_masks.squeeze()
            batch_min, batch_max = batch_min.squeeze(), batch_max.squeeze()

            
            pgnn_output = pgnn(batch_input, batch_dist, batch_dist_arg, batch_dist_mask)
            cnn_output = cnn(pgnn_output)
        
            batch_outputs = lstm(cnn_output)


            #----------------------------------------------------------------------
            Bangkok_vali_outputs = batch_outputs * (batch_max - batch_min) + batch_min
            Bangkok_vali_labels = batch_labels * (batch_max - batch_min) + batch_min

            loss_mse = criterion_MSE(Bangkok_vali_outputs, Bangkok_vali_labels) * batch_masks
            loss_mae = criterion_MAE(Bangkok_vali_outputs, Bangkok_vali_labels) * batch_masks
            loss_mse = loss_mse.sum() / batch_masks.sum()
            loss_mae = loss_mae.sum() / batch_masks.sum()    

            
        print(f"----Vali Set Bangkok: MSE={loss_mse.item():.1f}, MAE={loss_mae.item():.1f}")   


        for batch_input, batch_labels, batch_dist, batch_dist_arg, batch_masks, batch_min, batch_max, batch_dist_mask, _ in Bangkok_test_loader:
            batch_input, batch_labels, batch_masks = batch_input.squeeze(), batch_labels.squeeze(), batch_masks.squeeze()
            batch_min, batch_max = batch_min.squeeze(), batch_max.squeeze()

            
            pgnn_output = pgnn(batch_input, batch_dist, batch_dist_arg, batch_dist_mask)
            cnn_output = cnn(pgnn_output)
        
            batch_outputs = lstm(cnn_output)


            #----------------------------------------------------------------------
            Bangkok_test_outputs = batch_outputs * (batch_max - batch_min) + batch_min
            Bangkok_test_labels = batch_labels * (batch_max - batch_min) + batch_min

            loss_mse = criterion_MSE(Bangkok_test_outputs, Bangkok_test_labels) * batch_masks
            loss_mae = criterion_MAE(Bangkok_test_outputs, Bangkok_test_labels) * batch_masks
            loss_mse = loss_mse.sum() / batch_masks.sum()
            loss_mae = loss_mae.sum() / batch_masks.sum()    

            
        print(f"----Test Set Bangkok: MSE={loss_mse.item():.1f}, MAE={loss_mae.item():.1f}")   
        print("--------------------------------------------")


    if h == epoch_num-1: #epoch_num-1:
        pgnn_cnn_results = {"Antwerp_vali": Antwerp_vali_outputs.detach().numpy(), "Antwerp_vali_label": Antwerp_vali_labels.detach().numpy(), "Bangkok_vali": Bangkok_vali_outputs.detach().numpy(), "Bangkok_vali_label": Bangkok_vali_labels.detach().numpy(),
                    "Antwerp_test": Antwerp_test_outputs.detach().numpy(), "Antwerp_test_label": Antwerp_test_labels.detach().numpy(), "Bangkok_test": Bangkok_test_outputs.detach().numpy(), "Bangkok_test_label": Bangkok_test_labels.detach().numpy(),
                     "Antwerp_node_num": 1466,  "Bangkok_node_num": 1137}

        save_data(pgnn_cnn_results,"D:/ThesisData/processed data/SourceDomain/NEW/results/pgnn_cnn_rnn_results.h5" )    

    
    lstm_scheduler.step()
    cnn_pgnn_scheduler.step()
    cnn_scheduler.step()

    
        
            

epoch 1, Train Set Inversed Values: MSE=9031.9, MAE=71.0
----Validation Antwerp: MSE=3652.6, MAE=38.8
----Test Set Antwerp: MSE=5550.2, MAE=50.0
----Vali Set Bangkok: MSE=3186.4, MAE=38.9
----Test Set Bangkok: MSE=5128.8, MAE=52.6
--------------------------------------------
epoch 2, Train Set Inversed Values: MSE=6178.9, MAE=56.5
----Validation Antwerp: MSE=3396.2, MAE=37.4
----Test Set Antwerp: MSE=4995.0, MAE=47.6
----Vali Set Bangkok: MSE=2611.0, MAE=35.9
----Test Set Bangkok: MSE=4201.5, MAE=48.1
--------------------------------------------
epoch 3, Train Set Inversed Values: MSE=5147.3, MAE=49.7
----Validation Antwerp: MSE=3708.1, MAE=38.2
----Test Set Antwerp: MSE=5606.0, MAE=49.6
----Vali Set Bangkok: MSE=3094.4, MAE=38.5
----Test Set Bangkok: MSE=5198.9, MAE=54.0
--------------------------------------------
epoch 4, Train Set Inversed Values: MSE=5994.8, MAE=55.6
----Validation Antwerp: MSE=3366.5, MAE=36.9
----Test Set Antwerp: MSE=4957.5, MAE=47.0
----Vali Set Bangkok: MSE=2

In [47]:
#Save Model Para
torch.save({
    'pgnn_state_dict': pgnn.state_dict(),
    'rnn_state_dict': lstm.state_dict(),
    'cnn_state_dict': cnn.state_dict(),
    'rnn_optimizer_state_dict': lstm_optimizer.state_dict(),
    'cnn_pgnn_optimizer_state_dict': cnn_pgnn_optimizer.state_dict(),
    'cnn_optimizer_state_dict': cnn_optimizer.state_dict(),
}, 'D:/ThesisData/processed data/ModelPara/source_pgnn_cnn_lstm_14days.pth')